In [1]:
import logging
import sys

fx_spot_tickers = [
    "EURUSD=X",
    "GBPUSD=X",
    "JPYUSD=X",
    "AUDUSD=X",
    "NZDUSD=X",
    "USDCHF=X",
    "USDCAD=X",
    "USDSEK=X",
    "USDNOK=X",
]
logging.basicConfig(
    level=logging.INFO,
    stream=sys.stdout,
    format="[%(levelname)s] %(name)s: %(message)s",
    force=True,
)

In [2]:
import pickle as pkl
from importlib.resources import files

import yfinance as yf

import backtest_lib.examples

data = yf.download(
    tickers=fx_spot_tickers,
    start="2020-01-01",
    end="2025-01-01",
    interval="1d",
    auto_adjust=True,
)

pkl.dump(data, files(backtest_lib.examples).joinpath("fx_spot.pkl").open("wb"))

[*********************100%***********************]  9 of 9 completed


In [3]:
import backtest_lib as btl

px_flat = data["Close"].rename_axis(index="date", columns=None).ffill().reset_index()

market_view = btl.MarketView(prices=px_flat)

In [4]:
universe = tuple(fx_spot_tickers)


def index_strategy(universe, current_portfolio, market, ctx):
    return btl.Decision(current_portfolio)

In [5]:
from backtest_lib.backtest import Backtest
from backtest_lib.portfolio import uniform_portfolio

backtest = Backtest(
    market_view=market_view,
    universe=universe,
    strategy=index_strategy,
    initial_portfolio=uniform_portfolio(universe),
)

In [16]:
results = backtest.run()

In [17]:
results

BacktestResults(initial_capital=1.0, portfolio_returns=[0.0, 0.006426353015120683, -0.0012183887132028324, -0.00026479349525044253, -0.00039931328565146265, -0.00028395244124988816, -0.000265940550979754, -0.0010891077549663774, 0.0014228886685247494, -0.0017630250105327532, -0.0005110824595689118, -0.0002302336240864369, 0.0009922384164432819, -0.00040411188044850503, -0.00012870216460712477, 0.0008387055981693932, 0.0011364468221477133, 0.0005371818552093036, 0.0005055182129418457, 0.0005711244869196677, -0.0009479238112341794, 0.0007089203674570681, -0.00017610911683598065, 0.0009143202672783039, -0.0001544788979715234, -0.0009051139576148404, -0.0005867805395604471, -0.0007724871602541915, -0.00033759327982859353, -0.000222946785446935, -0.00024337698330771694, 0.00010648785171035154, 0.0013759471996021256, 0.000564504595475106, -0.0010539326790171644, 0.0009508159344428835, -0.003091450042230781, -0.0018025165782024936, -0.00011815917273768836, 0.001485343770938389, 0.000549161088

In [18]:
results.asset_returns.by_period

PolarsByPeriod(_period_column_df=shape: (9, 1_305)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ 2020-01-0 ┆ 2020-01-0 ┆ 2020-01-0 ┆ 2020-01-0 ┆ … ┆ 2024-12-2 ┆ 2024-12-2 ┆ 2024-12-3 ┆ 2024-12- │
│ 1 00:00:0 ┆ 2 00:00:0 ┆ 3 00:00:0 ┆ 6 00:00:0 ┆   ┆ 6 00:00:0 ┆ 7 00:00:0 ┆ 0 00:00:0 ┆ 31 00:00 │
│ 0.000000  ┆ 0.000000  ┆ 0.000000  ┆ 0.000000  ┆   ┆ 0.000000  ┆ 0.000000  ┆ 0.000000  ┆ :00.0000 │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ 00       │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0.0       ┆ 0.000358  ┆ -0.00489  ┆ -0.005868 ┆ … ┆ 0.00088   ┆ -0.003454 ┆ 0.000336  ┆ -0.00081 │
│           ┆           ┆           ┆   

In [19]:
results.asset_returns.by_security

PolarsBySecurity(_security_column_df=shape: (1_305, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ AUDUSD=X  ┆ EURUSD=X  ┆ GBPUSD=X  ┆ JPYUSD=X  ┆ … ┆ USDCAD=X  ┆ USDCHF=X  ┆ USDNOK=X  ┆ USDSEK=X │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0.0       ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 0.0      │
│ 0.000358  ┆ 0.0       ┆ -0.000928 ┆ -0.000294 ┆ … ┆ -0.00223  ┆ 0.00031   ┆ 0.049402  ┆ 0.018383 │
│ -0.00489  ┆ -0.004401 ┆ -0.007366 ┆ 0.001566  ┆ … ┆ 0.000771  ┆ 0.003824  ┆ 0.002387  ┆ 0.002356 │
│ -0.005868 ┆ -0.000848 ┆ -0.00552  ┆ 0.005363  ┆ … ┆ 0.000277  ┆ 0.000288  ┆ 0.004926  ┆ 0.005157 │
│ -0.000992 ┆ 0.003228  ┆ 0.006875  

In [20]:
results.asset_returns.by_security[["AUDUSD=X", "EURUSD=X"]].after(
    "2023-01-01"
).by_period[:5].by_security.to_dataframe()

date,AUDUSD=X,EURUSD=X
datetime[μs],f64,f64
2023-01-02 00:00:00,0.008439,0.004594
2023-01-03 00:00:00,-0.002564,-0.00299
2023-01-04 00:00:00,-0.011403,-0.012255
2023-01-05 00:00:00,0.015028,0.005642
2023-01-06 00:00:00,-0.01075,-0.007934
